In [2]:
%pip install -U langsmith
%pip install --upgrade --quiet  langchain-gigachat

  Using cached langsmith-0.2.1-py3-none-any.whl.metadata (14 kB)
Using cached langsmith-0.2.1-py3-none-any.whl (316 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.147
    Uninstalling langsmith-0.1.147:
      Successfully uninstalled langsmith-0.1.147
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.3.22 requires langsmith<0.2.0,>=0.1.125, but you have langsmith 0.2.1 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
from getpass import getpass

if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = ''

In [8]:
from langchain_gigachat import GigaChat

chat = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

In [11]:
from langchain_core.messages import HumanMessage, SystemMessage
from langsmith.wrappers import wrap_openai
from langsmith import traceable

@traceable
def start_dialog (input):
    messages = [
        SystemMessage(
            content= input
        ),
        HumanMessage(content="What is capital of Russia?"),
    ]

    print(chat.invoke(messages).content)

print(start_dialog ('кошка - '))

The capital city of Russia is Moscow.
None


In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="01cd58a7-eef9-4d4d-8f46-d914573d2088"

In [16]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

# 1. Create and/or select your dataset
client = Client()
dataset_name = "my first dataset"
dataset = client.clone_public_dataset("https://smith.langchain.com/public/a63525f9-bdf2-4512-83e3-077dc9417f96/d", dataset_name=dataset_name)

# 2. Define an evaluator
def is_concise_enough(root_run: Run, example: Example) -> dict:
    score = len(root_run.outputs["output"]) < 3 * len(example.outputs["answer"])
    return {"key": "is_concise", "score": int(score)}

# 3. Run an evaluation
# For more info on evaluators, see: https://docs.smith.langchain.com/concepts/evaluation#evaluators

# To evaluate an LCEL chain, replace lambda with chain.invoke
# To evaluate a LangGraph graph, replace lambda with graph.invoke
evaluate(
    lambda x: x["question"] + "is a good question. I don't know the answer.",
    # chain.invoke
    # graph.invoke
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="my first dataset experiment "
)

LangSmithError: Failed to GET /datasets in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/datasets?limit=1&name=my+first+dataset', '{"detail":"Forbidden"}')